In [1]:
from dataclasses import dataclass, field
from enum import Enum, auto
import random
import pandas as pd

In [2]:
# cards are numbers from 1 to 13
# the score is capped at 10

## Define a Hand class and functions on it

In [3]:
@dataclass
class Hand:
    """Class for representing a blackjack hand."""
    score: int = 0
    soft: bool = False
    cards: [int] = field(default_factory=list)


In [4]:
# For now, assume aces are always 1 TODO
def add_card(hand, card):
    new_score = min(hand.score+min(10, card), 22) # cap busted hands at 22
    hand.score = new_score
    hand.cards += [card]
    return hand

In [5]:
def is_busted(hand):
    return hand.score > 21

In [6]:
h = Hand()
h

Hand(score=0, soft=False, cards=[])

In [7]:
add_card(h, 6) # start a hand with a 6

Hand(score=6, soft=False, cards=[6])

In [8]:
add_card(h, 11) # show that J (11) counts as 10 points

Hand(score=16, soft=False, cards=[6, 11])

In [9]:
add_card(h, 7), is_busted(h) # bust (show that 23 is counted as 22)

(Hand(score=22, soft=False, cards=[6, 11, 7]), True)

## Now define gameplay and strategy

In [10]:

# TODO I might want a Flag class later, to provide a set of possible Actions
class Action(Enum):
    STAND = auto()
    HIT = auto()
    #DOUBLE = auto()
    #SPLIT = auto()
    
    

In [11]:
# Most simple/conservative strategy imaginable:
def strat_nobust(hand, dealer):
    if hand.score > 11:
        return Action.STAND
    else:
        return Action.HIT
        

In [12]:
# Dealer strategy
def strat_dealer(hand, dealer):
    if hand.score < 17:
        return Action.HIT
    # TODO handle soft hands
    else:
        return Action.STAND
        

In [13]:
class HandOutcome(Enum):
    WIN = 1
    LOSE = -1
    PUSH = 0

In [14]:
# Deck; completely random (i.e., infinite) for now


def deal_card():
    return random.randrange(13)+1

In [15]:
[deal_card() for _ in range(10)]

[9, 11, 10, 11, 4, 7, 7, 1, 3, 10]

In [16]:
# return the final hand after playing
def player_play_hand(strategy, hand, dealer, deck): 
    while True:
        decision = strategy(hand, dealer)
        if decision == Action.STAND:
            return hand
        if decision == Action.HIT:
            add_card(hand, deck())
            if is_busted(hand):
                return hand



In [17]:
def player_hand_outcome(player_hand, dealer_hand):
    # TODO blackjack outcome
    if is_busted(player_hand):
        return HandOutcome.LOSE
    if is_busted(dealer_hand):
        return HandOutcome.WIN
    if player_hand.score > dealer_hand.score:
        return HandOutcome.WIN
    if player_hand.score == dealer_hand.score:
        return HandOutcome.PUSH
    if player_hand.score < dealer_hand.score:
        return HandOutcome.LOSE


In [18]:
# One player and a dealer
# Player has a strategy
# Each gets dealt cards and plays according to strategy
# Emit all the data, and figure the rest out later

def play_one_hand(strat):
    hand_p = Hand()
    hand_d = Hand()

    add_card(hand_p, deal_card())
    add_card(hand_d, deal_card())
    add_card(hand_p, deal_card())
    dealer_hole_card = deal_card()
    
    # player
    player_play_hand(strat, hand_p, hand_d, deal_card)
    # dealer
    player_play_hand(strat_dealer, add_card(hand_d, dealer_hole_card), None, deal_card)
    
    return (hand_p, hand_d, player_hand_outcome(hand_p, hand_d))


play_one_hand(strat_nobust)

(Hand(score=14, soft=False, cards=[10, 4]),
 Hand(score=21, soft=False, cards=[1, 12, 13]),
 <HandOutcome.LOSE: -1>)

## Aggregate and summarize the data from the simulations

In [19]:

def generate_row_from_hand(h):
    (hand_p, hand_d, outcome) = h
    return {'hand_start': hand_p.cards[:2], 'dealer_card': hand_d.cards[0], 'hand_end': hand_p.cards, 'dealer_hand': hand_d.cards, 'outcome': outcome}

generate_row_from_hand(play_one_hand(strat_nobust))

{'hand_start': [12, 11],
 'dealer_card': 12,
 'hand_end': [12, 11],
 'dealer_hand': [12, 8],
 'outcome': <HandOutcome.WIN: 1>}

In [20]:
def run_n_sim_trials(strat, n):
    sims = pd.DataFrame([generate_row_from_hand(play_one_hand(strat)) for _ in range(n)])
    return sims

def summarize_totals(sims):
    return sims['outcome'].value_counts(), sims['outcome'].apply(lambda x: x.value).mean()

sims = run_n_sim_trials(strat_dealer, 1000)
sims, summarize_totals(sims)

(    hand_start  dealer_card        hand_end      dealer_hand           outcome
 0       [5, 4]            5      [5, 4, 13]    [5, 6, 5, 10]   HandOutcome.WIN
 1      [13, 5]            2      [13, 5, 3]  [2, 4, 5, 3, 4]  HandOutcome.PUSH
 2      [4, 11]            7      [4, 11, 4]        [7, 6, 7]  HandOutcome.LOSE
 3       [9, 8]            6          [9, 8]       [6, 1, 10]  HandOutcome.PUSH
 4       [8, 4]            7      [8, 4, 11]          [7, 13]  HandOutcome.LOSE
 ..         ...          ...             ...              ...               ...
 995   [12, 12]            6        [12, 12]        [6, 4, 7]   HandOutcome.WIN
 996     [3, 1]           13    [3, 1, 8, 9]       [13, 5, 9]   HandOutcome.WIN
 997    [13, 3]            4      [13, 3, 6]     [4, 4, 8, 4]  HandOutcome.LOSE
 998     [1, 5]            3  [1, 5, 13, 13]       [3, 5, 12]  HandOutcome.LOSE
 999     [8, 9]            6          [8, 9]       [6, 8, 11]   HandOutcome.WIN
 
 [1000 rows x 5 columns],
 (HandOutcom

In [21]:
def strat_simple(hand, dealer):
    if hand.score >= 17:  return Action.STAND
    if hand.score <= 11:  return Action.HIT
    if dealer.score in (range(3,7)):  return Action.STAND
    else:  return Action.HIT
        
sims = run_n_sim_trials(strat_simple, 1000)
sims, summarize_totals(sims)

(    hand_start  dealer_card         hand_end    dealer_hand           outcome
 0      [9, 12]           10          [9, 12]    [10, 4, 12]   HandOutcome.WIN
 1       [2, 3]            7    [2, 3, 4, 12]     [7, 3, 10]  HandOutcome.LOSE
 2      [2, 13]           12       [2, 13, 5]       [12, 11]  HandOutcome.LOSE
 3       [8, 8]            6           [8, 8]      [6, 5, 8]  HandOutcome.LOSE
 4      [10, 6]           13       [10, 6, 3]     [13, 4, 6]  HandOutcome.LOSE
 ..         ...          ...              ...            ...               ...
 995     [5, 8]           10        [5, 8, 4]  [10, 3, 1, 9]   HandOutcome.WIN
 996   [11, 13]           13         [11, 13]       [13, 10]  HandOutcome.PUSH
 997   [10, 11]            3         [10, 11]     [3, 11, 7]  HandOutcome.PUSH
 998     [1, 5]            7  [1, 5, 2, 8, 3]     [7, 7, 10]   HandOutcome.WIN
 999     [7, 6]           12     [7, 6, 3, 7]       [12, 11]  HandOutcome.LOSE
 
 [1000 rows x 5 columns],
 (HandOutcome.LOSE    47

In [22]:
for strat in [strat_dealer, strat_nobust, strat_simple]:
    print(summarize_totals(run_n_sim_trials(strat, 1000000)))


(HandOutcome.LOSE    500155
HandOutcome.WIN     400893
HandOutcome.PUSH     98952
Name: outcome, dtype: int64, -0.099262)
(HandOutcome.LOSE    512519
HandOutcome.WIN     424405
HandOutcome.PUSH     63076
Name: outcome, dtype: int64, -0.088114)
(HandOutcome.LOSE    488093
HandOutcome.WIN     422665
HandOutcome.PUSH     89242
Name: outcome, dtype: int64, -0.065428)
